In [1]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

# System Recording

If you need to focus on a smaller part of your model, it may be useful to attach a case recorder to a particular `System`. There are slightly different options when recording from these objects. System recording can only be used in serial running environments and cannot currently be used in parallel.

In [2]:
om.show_options_table("openmdao.core.system.System", recording_options=True)

Option,Default,Acceptable Values,Acceptable Types,Description
excludes,[],N/A,['list'],Patterns for vars to exclude in recording (processed post-includes). Uses fnmatch wildcards
includes,['*'],N/A,['list'],Patterns for variables to include in recording. Uses fnmatch wildcards
options_excludes,[],N/A,['list'],User-defined metadata to exclude in recording
record_inputs,True,"[True, False]",['bool'],Set to True to record inputs at the system level
record_metadata,True,"[True, False]",['bool'],Deprecated. Recording of metadata will always be done
record_model_metadata,True,"[True, False]",['bool'],Deprecated. Recording of model metadata will always be done
record_outputs,True,"[True, False]",['bool'],Set to True to record outputs at the system level
record_residuals,True,"[True, False]",['bool'],Set to True to record residuals at the system level


```{note}
Note that the `excludes` option takes precedence over the `includes` option.
```

## System Recording Example

In [3]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

recorder = om.SqliteRecorder("cases.sql")

obj_cmp = prob.model.obj_cmp
obj_cmp.add_recorder(recorder)
obj_cmp.recording_options['includes'] = ['*']
obj_cmp.recording_options['excludes'] = ['obj_cmp.x']

prob.model.nonlinear_solver.options['use_apply_nonlinear'] = True

prob.run_model()

NL: NLBGS Converged in 7 iterations


In [4]:
prob.cleanup()

cr = om.CaseReader("cases.sql")

system_cases = cr.list_cases('root.obj_cmp')

system
rank0:root._solve_nonlinear|0|NonlinearBlockGS|0|obj_cmp._solve_nonlinear|0


system
rank0:root._solve_nonlinear|0|NonlinearBlockGS|1|obj_cmp._solve_nonlinear|1


system
rank0:root._solve_nonlinear|0|NonlinearBlockGS|2|obj_cmp._solve_nonlinear|2


system
rank0:root._solve_nonlinear|0|NonlinearBlockGS|3|obj_cmp._solve_nonlinear|3


system
rank0:root._solve_nonlinear|0|NonlinearBlockGS|4|obj_cmp._solve_nonlinear|4


system
rank0:root._solve_nonlinear|0|NonlinearBlockGS|5|obj_cmp._solve_nonlinear|5


system
rank0:root._solve_nonlinear|0|NonlinearBlockGS|6|obj_cmp._solve_nonlinear|6


In [5]:
assert len(system_cases) == 7

In [6]:
case = cr.get_case(system_cases[0])

case.inputs

{'y1': array([27.8]), 'y2': array([12.27257053]), 'z': array([5., 2.])}

In [7]:
assert sorted(case.inputs.keys()) == ['y1', 'y2', 'z']